In [9]:
import pandas as pd
import json
import re

# Users JSON
with open('users.json', 'r') as f:
    users_df = pd.DataFrame(json.load(f))

# Orders CSV
orders_df = pd.read_csv('orders.csv')
orders_df['order_date'] = pd.to_datetime(orders_df['order_date'], dayfirst=True)

# SQL file 
restaurants_data = []
with open('restaurants.sql', 'r') as f:
    for line in f:
        if "INSERT INTO restaurants VALUES" in line:
            # Using Regex to extract content inside the parentheses
            match = re.search(r'\((.*)\)', line)
            if match:
                # Handle splitting while ignoring commas inside quoted strings
                parts = re.findall(r"'[^']*'|[^,]+", match.group(1))
                rid = int(parts[0].strip())
                name = parts[1].strip().strip("'")
                cuisine = parts[2].strip().strip("'")
                rating = float(parts[3].strip())
                restaurants_data.append([rid, name, cuisine, rating])

restaurants_df = pd.DataFrame(restaurants_data, columns=['restaurant_id', 'restaurant_name_sql', 'cuisine', 'rating'])

# Merge files

# Combine all three datasets  
merged_df = orders_df.merge(users_df, on='user_id').merge(restaurants_df, on='restaurant_id')

#for Gold Members
gold_df = merged_df[merged_df['membership'] == 'Gold']

# for MCQ 

# Q1
mcq1_city = gold_df.groupby('city')['total_amount'].sum().idxmax()

# Q2
mcq2_cuisine = merged_df.groupby('cuisine')['total_amount'].mean().idxmax()

# Q3
user_spend = merged_df.groupby('user_id')['total_amount'].sum()
mcq3_count = (user_spend > 1000).sum() # Result: 2544 (Fits "> 2000" option)

# Q4
bins = [3.0, 3.6, 4.1, 4.6, 5.1]
labels = ['3.0 – 3.5', '3.6 – 4.0', '4.1 – 4.5', '4.6 – 5.0']
merged_df['rating_range'] = pd.cut(merged_df['rating'], bins=bins, labels=labels, right=False)
mcq4_range = merged_df.groupby('rating_range')['total_amount'].sum().idxmax()

# Q5
mcq5_avg_city = gold_df.groupby('city')['total_amount'].mean().idxmax()

# Q6
mcq6_cuisine = merged_df.groupby('cuisine')['restaurant_id'].nunique().idxmin()

# Q7
mcq7_pct = round((len(gold_df) / len(merged_df)) * 100)

# Q8
res_stats = merged_df.groupby('restaurant_name').agg(count=('order_id', 'count'), avg_omv=('total_amount', 'mean'))
options_8 = ['Grand Cafe Punjabi', 'Grand Restaurant South Indian', 'Ruchi Mess Multicuisine', 'Ruchi Foods Chinese']
mcq8_res = res_stats[(res_stats.index.isin(options_8)) & (res_stats['count'] < 20)]['avg_omv'].idxmax()

# Q9
combos = [('Gold', 'Indian'), ('Gold', 'Italian'), ('Regular', 'Indian'), ('Regular', 'Chinese')]
combo_rev = merged_df.groupby(['membership', 'cuisine'])['total_amount'].sum()
mcq9_combo = combo_rev[combo_rev.index.isin(combos)].idxmax()

# Q10
merged_df['quarter'] = merged_df['order_date'].dt.quarter
mcq10_qtr = f"Q{merged_df.groupby('quarter')['total_amount'].sum().idxmax()}"

# for numericals 
num1_gold_orders = len(gold_df)
num2_hyd_rev = round(merged_df[merged_df['city'] == 'Hyderabad']['total_amount'].sum())
num3_unique_users = merged_df['user_id'].nunique()
num4_avg_gold_val = round(gold_df['total_amount'].mean(), 2)
num5_high_rating = len(merged_df[merged_df['rating'] >= 4.5])
num6_top_city_gold_orders = len(gold_df[gold_df['city'] == mcq1_city])

# Print results 
print("MULTIPLE CHOICE ANSWERS")
print(f"1. City has the highest total revenue: {mcq1_city}")
print(f"2. Highest avg order Cuisine: {mcq2_cuisine}")
print(f"3. Users orders more than rs.1000: {mcq3_count} ")
print(f"4. Highest Rating Range: {mcq4_range}")
print(f"5. Gold members highest avg order value: {mcq5_avg_city}")
print(f"6. Cuisine with lowest restaurants: {mcq6_cuisine}")
print(f"7. Order Percentage of Gold members: {mcq7_pct}%")
print(f"8. Highest avg order value less than 20 orders: {mcq8_res}")
print(f"9. Highest Revenue Combination: {mcq9_combo}")
print(f"10. Highest Revenue Quarter: {mcq10_qtr}")

print("\nNUMERICAL ANSWERS")
print(f"1. Total Gold Orders: {num1_gold_orders}")
print(f"2. Hyderabad Revenue: {num2_hyd_rev}")
print(f"3. Distinct Users: {num3_unique_users}")
print(f"4. Avg Gold Order Value: {num4_avg_gold_val}")
print(f"5. Rating >= 4.5 Orders: {num5_high_rating}")
print(f"6. Orders in top Gold city: {num6_top_city_gold_orders}")

MULTIPLE CHOICE ANSWERS
1. City has the highest total revenue: Chennai
2. Highest avg order Cuisine: Mexican
3. Users orders more than rs.1000: 2544 
4. Highest Rating Range: 4.6 – 5.0
5. Gold members highest avg order value: Chennai
6. Cuisine with lowest restaurants: Chinese
7. Order Percentage of Gold members: 50%
8. Highest avg order value less than 20 orders: Ruchi Foods Chinese
9. Highest Revenue Combination: ('Gold', 'Italian')
10. Highest Revenue Quarter: Q3

NUMERICAL ANSWERS
1. Total Gold Orders: 4987
2. Hyderabad Revenue: 1889367
3. Distinct Users: 2883
4. Avg Gold Order Value: 797.15
5. Rating >= 4.5 Orders: 3374
6. Orders in top Gold city: 1337
